### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
2019-03-11
Azure Machine Learning SDK for Python v1.0.18  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v.1.0.30' # also set this in conda file below

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v.1.0.30'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.6 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain]==1.0.30 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

FROM continuumio/miniconda3:4.5.12
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli        


ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH         
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v.1.0.30


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + image_version_string


docker_file_path
docker_image_name
docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v.1.0.30'

'georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.30'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm001 loginvm001  313 Apr 27 02:40 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm001 loginvm001 1228 Apr 27 02:40 dockerfile_aml-sdk_docker_image_sdk.v.1.0.30


In [9]:
!echo docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache
!     docker build -t $docker_image_name -f $docker_file_path $working_path --no-cache

docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.30 -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v.1.0.30 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache
Sending build context to Docker daemon  4.096kB
Step 1/14 : FROM continuumio/miniconda3:4.5.12
 ---> ae46c364060f
Step 2/14 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in 95ae8775f81d
Removing intermediate container 95ae8775f81d
 ---> 59746297716d
Step 3/14 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in 30783e244c74
Ign:1 http://deb.debian.org/debian stretch InRelease
Get:2 http://deb.debian.org/debian stretch-updates InRelease [91.0 kB]
Get:3 http://deb.debian.org/debian stretch Release [118 kB]
Get:4 http://deb.debian.org/debian stretc

Get:50 http://deb.debian.org/debian stretch/main amd64 libstdc++-6-dev amd64 6.3.0-18+deb9u1 [1420 kB]
Get:51 http://deb.debian.org/debian stretch/main amd64 g++-6 amd64 6.3.0-18+deb9u1 [7094 kB]
Get:52 http://deb.debian.org/debian stretch/main amd64 g++ amd64 4:6.3.0-4 [1546 B]
Get:53 http://deb.debian.org/debian stretch/main amd64 lsb-release all 9.20161125 [27.1 kB]
Get:54 http://deb.debian.org/debian stretch/main amd64 make amd64 4.1-9.1 [302 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 50.7 MB in 0s (102 MB/s)
(Reading database ... 10360 files and directories currently installed.)
Preparing to unpack .../libc6_2.24-11+deb9u4_amd64.deb ...
debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline
Unpacking libc6:amd64 (2.24-11+deb9u4) over (2.24-11+deb9u3) ...
Setting up libc6:amd64 (2.24-11+deb9u4) ...
debconf: unable to initialize frontend: D

Selecting previously unselected package gcc-6.
Preparing to unpack .../30-gcc-6_6.3.0-18+deb9u1_amd64.deb ...
Unpacking gcc-6 (6.3.0-18+deb9u1) ...
Selecting previously unselected package gcc.
Preparing to unpack .../31-gcc_4%3a6.3.0-4_amd64.deb ...
Unpacking gcc (4:6.3.0-4) ...
Selecting previously unselected package libc-dev-bin.
Preparing to unpack .../32-libc-dev-bin_2.24-11+deb9u4_amd64.deb ...
Unpacking libc-dev-bin (2.24-11+deb9u4) ...
Selecting previously unselected package linux-libc-dev:amd64.
Preparing to unpack .../33-linux-libc-dev_4.9.144-3.1_amd64.deb ...
Unpacking linux-libc-dev:amd64 (4.9.144-3.1) ...
Selecting previously unselected package libc6-dev:amd64.
Preparing to unpack .../34-libc6-dev_2.24-11+deb9u4_amd64.deb ...
Unpacking libc6-dev:amd64 (2.24-11+deb9u4) ...
Selecting previously unselected package libstdc++-6-dev:amd64.
Preparing to unpack .../35-libstdc++-6-dev_6.3.0-18+deb9u1_amd64.deb ...
Unpacking libstdc++-6-dev:amd64 (6.3.0-18+deb9u1) ...
Selecting prev

  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
  Stored in directory: /root/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
  Stored in directory: /root/.cache/pip/wheels/45/cb/7e/ce6e6062c69446e39e328170524ca8213498bc66a74c6a210b
  Stored in directory: /root/.cache/pip/wheels/5b/d7/0f/e58eae695403de585269f4e4a94e0cd6ca60ec0c202936fa4a
  Stored in directory: /root/.cache/pip/wheels/bf/26/bd/912db1314f1cef0171d9b7f128dd01e8b8c92ed8d0062e632d
  Stored in directory: /root/.cache/pip/wheels/f2/9a/90/de94f8556265ddc9d9c8b271b0f63e57b26fb1d67a45564511
  Stored in directory: /root/.cache/pip/wheels/ee/30/5c/3a3b5e1386c8db9a3be5f5c3933644ae0533c1351c6a8eb4b5
  Stored in directory: /root/.cache/pip/wheels/e8/74/51/42c2d41c02bdc6f0e604476b7e4293b8c98d0bcbfa1dff78c8
  Stored in directory: /root/.cache/pip/wheels/b8/cb/43/c0ba52baf2b0e371ec1d5b2d4685d6d24617b1391f3eeacda5
  Stored in directory: /root/.cache/p

  Found existing installation: nbconvert 5.4.1
    Uninstalling nbconvert-5.4.1:
      Successfully uninstalled nbconvert-5.4.1
  Found existing installation: scipy 1.2.1
    Uninstalling scipy-1.2.1:
      Successfully uninstalled scipy-1.2.1
  Found existing installation: wheel 0.33.1
    Uninstalling wheel-0.33.1:
      Successfully uninstalled wheel-0.33.1
Cache location: /opt/conda/pkgs
Will remove the following tarballs:

/opt/conda/pkgs
---------------
jedi-0.13.3-py36_0.tar.bz2                   236 KB
sqlite-3.28.0-h7b6447c_0.tar.bz2             1.9 MB
pickleshare-0.7.5-py36_0.tar.bz2              13 KB
numpy-base-1.16.3-py36hde5b4d6_0.tar.bz2     4.4 MB
pygments-2.3.1-py36_0.tar.bz2                1.4 MB
notebook-5.7.8-py36_0.tar.bz2                7.3 MB
six-1.12.0-py36_0.tar.bz2                     22 KB
readline-7.0-h7b6447c_5.tar.bz2              392 KB
defusedxml-0.6.0-py_0.tar.bz2                 23 KB
jupyter_core-4.4.0-py36_0.tar.bz2             63 KB
nb_conda-2.2.1-p

Removing intermediate container b03ce199a1c0
 ---> e7cb58653b2e
Step 10/14 : ENV CONDA_AUTO_UPDATE_CONDA=false
 ---> Running in 090443833eb9
Removing intermediate container 090443833eb9
 ---> 9ce951127ff0
Step 11/14 : ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
 ---> Running in 98fe930dc487
Removing intermediate container 98fe930dc487
 ---> 1445f5c9524c
Step 12/14 : ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
 ---> Running in 4222501dcd0d
Removing intermediate container 4222501dcd0d
 ---> 3b4ea6a57e41
Step 13/14 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in 31d605cfcbdc
Removing intermediate container 31d605cfcbdc
 ---> 682fc1473d89
Step 14/14 : CMD /bin/bash
 ---> Running in d1b2d64d028e
Removing intermediate container d1b2d64d028e
 ---> f46253d2f958
Successfully built f46253d2f958
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.30


In [10]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir'

total 8
-rw-r--r-- 1 root root 5117 Apr 18 03:14 exported_conda_env.yml


In [11]:
# export the resulted conda env
!docker run  \
        -v $crt_local_dir:/workspace:rw \
        $docker_image_name \
        /bin/bash -c "conda env export > /workspace/exported_conda_env.yml"

In [12]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [13]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.30 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [14]:
!echo docker run  $docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $docker_image_name /bin/bash -c "az --version"

docker run georgedockeraccount/aml-sdk_docker_image:sdk.v.1.0.30 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.30
azure-cli                         2.0.63

acr                                2.2.5
acs                               2.3.22
advisor                            2.0.0
ams                                0.4.5
appservice                        0.2.18
backup                             1.2.4
batch                              4.0.0
batchai                            0.4.8
billing                            0.2.1
botservice                        0.1.10
cdn                                0.2.3
cloud                              2.1.1
cognitiveservices                  0.2.5
command-modules-nspkg               2.0.2
configure                         2.0.22
consumption                        0.4.2
container                         0.3.16
core                              2.0.63
cosmosdb                          0.2.10
deploymentmanager                

#### Do the same in a  notebook

In [15]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [16]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir/test_aml_sdk_docker_image.ipynb


In [17]:
!ls -l $crt_local_dir

total 12
-rw-r--r-- 1 root       root       5085 Apr 27 02:49 exported_conda_env.yml
-rw-rw-r-- 1 loginvm001 loginvm001  982 Apr 27 02:49 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [18]:
!docker run -v $crt_local_dir:/workspace:rw $docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [19]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.30\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.8"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [20]:
# !docker login -u=$docker_login -p=
!docker push $docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

18b75d81: Preparing 
07f4b2bf: Preparing 
82ee667d: Preparing 
8148584f: Preparing 
6384d46a: Preparing 
8fbc5764: Preparing 
828fac1a: Preparing 
ae060f2d: Preparing 


18b75d81: Pushing  1.616GB/2.12GBBAPushing  738.5MB/2.12GBPushing  1.197GB/2.12GB

18b75d81: Pushed   2.173GB/2.12GBPushing  1.924GB/2.12GBsdk.v.1.0.30: digest: sha256:07f409f272853d5f3d06918a912c6b7811eb73459a6575bf5225bf732f75a233 size: 2214


In [21]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 370009 bytes to createAMLSDKDocker.html
